[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab10/lab10.ipynb)

# Práctica 10: Autoencoders

## Pre-requisitos

### Instalar paquetes

Si la práctica requiere algún paquete de Python, habrá que incluir una celda en la que se instalen. Si usamos un paquete que se ha utilizado en prácticas anteriores, podríamos dar por supuesto que está instalado pero no cuesta nada satisfacer todas las dependencias en la propia práctica para reducir las dependencias entre ellas.

### NOTA: En <font color='red'>Google Colab</font> hay que instalar los paquetes EN CADA EJECUCIÓN

In [ ]:
# Ejemplo de instalación de tensorflow 2.0
#%tensorflow_version 2.x
# !pip3 install tensorflow  # NECESARIO SOLO SI SE EJECUTA EN LOCAL
import tensorflow as tf

# Hacemos los imports que sean necesarios
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import sklearn

# Autoencoders

El autoencoder es un tipo de red que se utiliza para aprender codificaciones eficientes de datos sin etiquetar (lo que se conoce como aprendizaje no supervisado). Es una red que tiene el mismo tamaño en la entrada como en la salida, puesto que el objetivo de la red es reconstruír la entrada con la menor pérdida posible.

Si lo que hacemos es reconstruír la entrada, ¿qué sentido tiene el usar la red? Habitualmente, **la red consta, a su mitad, de una capa con menos elementos que los datos de entrada**. Por tanto, al reconstruír los datos de la entrada a la salida, en esa capa tendremos una versión *comprimida* de la entrada, que contendrá la mayor parte de su información.

Por tanto, podemos dividir un autoencoder en 3 secciones diferentes, tal y como se ve en la siguiente figura:

![](https://drive.google.com/uc?export=view&id=1yxkKZV0J0YplQAGPGJxQ2Z80Ad6L94eu)

1. **Encoder:** es la parte inicial de la red, encargada de comprimir los datos de la entrada.
1. **Code:** es la salida del encoder, contiene la versión *comprimida* de los datos de entrada.
1. **Decoder:** se encarga de, partiendo de la salida del *Encoder*, reconstruír la red.

# Autoencoders sobre MNIST

Lo que queremos hacer es entrenar un autoencoder para que nos proporcione una versión comprimida del dataset MNIST. Para ello, lo primero que tenemos que hacer es cargar el dataset.

In [ ]:
_, (x_train, y_train), = tf.keras.datasets.mnist.load_data()

# Haz el preprocesado que necesites aquí (si lo necesitas)

## Crea tu propio Autoencoder

Estamos a final de curso, ya deberías de ser capaz de realizar este trabajo con muy poca ayuda. ¡Es hora de que pongas en práctica todos los conocimientos de la asignatura! El diseño del autoencoder es libre (capas densas, convolucionales, ...), puedes crearlo como quieras. **El único parámetro fijo es la salida del encoder: tendrá un tamaño de 10 unidades.**


In [ ]:
# TODO: crea tu propio encoder
encoder = None

In [ ]:
# TODO: crea tu propio decoder. Habitualmente sigue la misma configuración del encoder, en sentido inverso.
decoder = None

In [ ]:
# TODO: crea el autoencoder como una combinación del encoder y el decoder. 
# RECUERDA: en keras se puede hacer un modelo secuencial que contenga otros modelos
autoencoder = None

In [ ]:
# TODO: compila y entrena la red
None

# Aprendizaje no supervisado

Como comentábamos al inicio, los autoencoder se utilizan para el aprendizaje no supervisado, esto es, cuando no tenemos etiquetados los datos. En el dataset MNIST los tenemos etiquetados, pero vamos a operar como si no fuera así, para finalmente comprobar cómo de bien hemos agrupado los datos.

**Antes de nada, obtén la representación comprimida de los datos.**

**<font color='red'>PISTA:</font> si has creado correctamente la red, podrás obtener la representación simplemente obteniendo la salida del encoder.**

In [ ]:
# TODO: obtén la representación comprimida de los datos
autoencoder_data = None

A modo de comparación, vamos a utilizar otro modelo no supervisado para obtener una representación de los datos. En este caso, haremos uso de un viejo conocido: el [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

In [ ]:
from sklearn.decomposition import PCA
pca_data = PCA(n_components=10).fit_transform(x_train.reshape((len(x_train), -1)))

#### Visualización de los datos con t-SNE

[t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) es una herramienta para visualizar datos en alta dimensionalidad. No vamos a entrar en detalle sobre cómo funciona, sólo tiene que quedar clara una cuestión: **cuanto más diferenciadas estén los clústers pertenecientes a una misma clase, mejor compresión de datos tendremos.**

In [ ]:
from sklearn.manifold import TSNE

tsne_data = {
             'PCA': TSNE(n_components=2, init='random').fit_transform(pca_data),
             'Autoencoder': TSNE(n_components=2, init='random').fit_transform(autoencoder_data)
}

In [ ]:
# usaremos esta función para visualizar los resultados
from sklearn.preprocessing import MinMaxScaler

def plot_embedding(X, y, title, ax):
    X = MinMaxScaler().fit_transform(X)

    shown_images = np.array([[1.0, 1.0]])  # just something big
    for i in range(X.shape[0]):
        # plot every digit on the embedding
        ax.text(
            X[i, 0],
            X[i, 1],
            str(y[i]),
            color=plt.cm.Dark2(y[i]),
            fontdict={"weight": "bold", "size": 9},
        )


    ax.set_title(title)
    ax.axis("off")


fig, axs = plt.subplots(nrows=1, ncols=2)
names = list(tsne_data.keys())

for name, ax in zip(names, axs.ravel()):
    if name is None:
        ax.axis("off")
        continue
    title = f"{name}"
    plot_embedding(tsne_data[name], y_train, title, ax)

plt.show()



Deberías poder ver una buena separación entre los números. Si tu autoencoder no mejora los resultados de PCA, deberás mejorarlo!

## Evaluación empírica

Ya hemos visto visualmente cómo de buena es nuestra compresión, ahora toca verlo de manera empírica. Para ello, vamos a utilizar uno de los algoritmos de clústering más clásicos: [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)


In [ ]:
from sklearn.cluster import KMeans

pca_kmeans = KMeans(n_clusters=10, random_state=26).fit_predict(pca_data)
autoencoder_kmeans = KMeans(n_clusters=10, random_state=26).fit_predict(autoencoder_data)

Para evaluar cómo de bien lo ha hecho, hay que tener en cuenta que, al ser no supervisado, puede asignar etiquetas distintas para las que tenemos en nuestro dataset. Por ello, necesitamos usar una función para asignar la correspondencia **etiqueta dataset - etiqueta clúster**. En nuestro caso, vamos a utilizar el *Normalized Mutual Info Score*.

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score

print('PCA NMI:', normalized_mutual_info_score(pca_kmeans, y_train))
print('Autoencoder NMI:', normalized_mutual_info_score(autoencoder_kmeans, y_train))

### Si el NMI de tu autoencoder es inferior al del PCA, ¡trabaja para mejorarlo!

# ¡ENHORABUENA! Has completado la práctica de Autoencoders.

# ¿Deseas saber más?

Los autoencoders son una herramienta muy potente, no sólo para la compresión de datos. Por ejemplo, también se utilizan para la generación de datos artificiales, con los conocidos *Variational Autoencoders* [VAE](https://www.youtube.com/watch?v=Q1XuXwPVFko).

El aprendizaje no supervisado no se limita sólo a clústering. Actualmente está en auge una disciplina llamada **Self-supervised learning**, que nos permite hacer cosas sin supervisar increíbles hace menos de un año, tales como [DINO](https://www.youtube.com/watch?v=8I1RelnsgMw), creado por Facebook.

# Trabajo extra

¿Has probado a hacer el autoencoder totalmente convolucional? Para el *decoder* puedes usar las funciones [UpSampling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling2D) o [Conv2DTranspose](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose).

In [ ]:
# TODO: escribe el código para el trabajo extra sin ayuda. Usa todos los bloques de código que quieras.